<a href="https://colab.research.google.com/github/aryanshu/NLP/blob/master/bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
cd "/gdrive/My Drive/Colab Notebooks/"

/gdrive/My Drive/Colab Notebooks


In [0]:

import pandas as pd
import numpy as np
import pickle

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score


In [0]:
!pip install pytorch-pretrained-bert pytorch-nlp


In [0]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np


In [0]:
"""
train_file_name = './datasets-v5/task-1/task1.train.txt'
train_file_name
"""

"\ntrain_file_name = './datasets-v5/task-1/task1.train.txt'\ntrain_file_name\n"

In [0]:
"""
articles_id, articles_content, gold_labels = ([], [], [])
with open(train_file_name, "r", encoding='utf-8') as f:
    for line in f.readlines():
        article_content, article_id, gold_label = line.rstrip().split("\t")
        articles_id.append(article_id)
        articles_content.append(article_content)
        gold_labels.append(gold_label)
print("Number of documents in the training set: %d"%(len(articles_content)))
"""

'\narticles_id, articles_content, gold_labels = ([], [], [])\nwith open(train_file_name, "r", encoding=\'utf-8\') as f:\n    for line in f.readlines():\n        article_content, article_id, gold_label = line.rstrip().split("\t")\n        articles_id.append(article_id)\n        articles_content.append(article_content)\n        gold_labels.append(gold_label)\nprint("Number of documents in the training set: %d"%(len(articles_content)))\n'

In [0]:
#train = pd.DataFrame({'id':articles_id, 'text': articles_content, 'target': gold_labels})

In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [0]:
"""
train["target"][train["target"]=="non-propaganda"]=0
train["target"][train["target"]=="propaganda"]=1
train.head()
"""

'\ntrain["target"][train["target"]=="non-propaganda"]=0\ntrain["target"][train["target"]=="propaganda"]=1\ntrain.head()\n'

In [0]:
#train.target=train.target.astype(int)

In [0]:
"""
train_df_bert = pd.DataFrame({
    'id':range(len(train)),
    'label':train.iloc[:,2],
    'alpha':['a']*train.shape[0],
    'text': train.iloc[:,1].replace(r'\n', ' ', regex=True)
})

train_df_bert.head()
"""

"\ntrain_df_bert = pd.DataFrame({\n    'id':range(len(train)),\n    'label':train.iloc[:,2],\n    'alpha':['a']*train.shape[0],\n    'text': train.iloc[:,1].replace(r'\n', ' ', regex=True)\n})\n\ntrain_df_bert.head()\n"

In [0]:
#train_df_bert.to_csv('train.tsv', sep='\t', index=False, header=False)

In [0]:
df = pd.read_csv("train.tsv", delimiter='\t', header=None, names=['sentence_source', 'label', 'label_notes', 'sentence'])

In [0]:
df.head()

,sentence_source,label,label_notes,sentence
0,0,0,a,"Et tu, Rhody? A recent editorial in the Provi..."
1,1,0,a,A recent post in The Farmington Mirror — our t...
2,2,0,a,"President Donald Trump, as he often does while..."
3,3,0,a,"February is Black History Month, and nothing l..."
4,4,0,a,"The snow was so heavy, whipped up by gusting w..."


In [0]:
sentences = df.sentence.values

sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences]
labels = df.label.values

In [0]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print ("Tokenize the first sentence:")
print (tokenized_texts[0])

Tokenize the first sentence:
['[CLS]', 'et', 'tu', ',', 'r', '##ho', '##dy', '?', 'a', 'recent', 'editorial', 'in', 'the', 'providence', 'journal', 'catalog', '##ed', 'everything', 'it', 'could', 'find', 'wrong', 'with', 'connecticut', 'and', 'ended', 'with', 'this', 'suggestion', ':', '“', 'gov', '.', 'gina', 'rai', '##mond', '##o', 'should', 'see', 'if', 'at', 'least', 'some', 'of', 'those', 'jobs', 'could', 'come', 'to', 'rhode', 'island', '.', 'it', 'is', 'certainly', 'less', 'risky', 'than', 'the', 'nut', '##me', '##g', 'state', '.', '”', 'we', 'beg', 'your', 'pardon', '.', 'the', 'state', 'with', 'world', '-', 'famous', 'pension', 'problems', 'and', 'persistent', 'economic', 'issues', 'of', 'its', 'own', 'is', '“', 'less', 'risky', '”', '?', 'the', 'journal', 'itself', 'reported', 'just', 'a', 'few', 'weeks', 'ago', 'on', 'rhode', 'island', '’', 's', 'own', 'significant', 'economic', 'problems', ',', 'which', 'in', 'many', 'ways', 'reflect', 'connecticut', '’', 's', '.', 'rhode',

In [0]:
for i,x in enumerate(tokenized_texts):
  if len(x)>512:
    tokenized_texts[i]=tokenized_texts[i][:511]
    

In [0]:

tokenized_texts = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]


In [0]:
tokenized_texts = pad_sequences(tokenized_texts, dtype="long", truncating="post", padding="post")

In [0]:
attention_masks = []

for seq in tokenized_texts:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)

In [0]:
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(tokenized_texts, labels, 
                                                            random_state=2018, test_size=0.2)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, tokenized_texts,
                                             random_state=2018, test_size=0.2)

In [0]:
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [0]:

batch_size = 4



train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)


In [0]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
   

In [0]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

In [0]:
optimizer = BertAdam(optimizer_grouped_parameters,
                     lr=2e-5,
                     warmup=.1)


t_total value of -1 results in schedule not being applied


In [0]:
from torch.nn import CrossEntropyLoss
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [0]:
# Store our loss and accuracy for plotting
train_loss_set = []

# Number of training epochs (authors recommend between 2 and 4)
epochs = 4

# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc="Epoch"):
  
  
  # Training
  
  # Set our model to training mode (as opposed to evaluation mode)
  model.train()
  
  # Tracking variables
  tr_loss = 0
  nb_tr_examples, nb_tr_steps = 0, 0
  
  # Train the data for one epoch
  for step, batch in enumerate(train_dataloader):
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_tokenized_texts, b_input_mask, b_labels = batch
    # Clear out the gradients (by default they accumulate)
    optimizer.zero_grad()
    # Forward pass
    logits = model(b_tokenized_texts, token_type_ids=None, attention_mask=b_input_mask)
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy() 
    # Backward pass
    loss_fct = CrossEntropyLoss()
    loss = loss_fct(logits.view(-1, 2), label_ids.view(-1))
    train_loss_set.append(loss.item()) 
    loss.backward()
    # Update parameters and take a step using the computed gradient
    optimizer.step()
    if step%20==0:
      break
    
    # Update tracking variables
    tr_loss += loss.item()
    nb_tr_examples += b_tokenized_texts.size(0)
    nb_tr_steps += 1

  print("Train loss: {}".format(tr_loss/nb_tr_steps))
    
    
  # Validation

  # Put model in evaluation mode to evaluate loss on the validation set
  model.eval()

  # Tracking variables 
  eval_loss, eval_accuracy = 0, 0
  nb_eval_steps, nb_eval_examples = 0, 0

  # Evaluate data for one epoch
  for batch in validation_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_tokenized_texts, b_input_mask, b_labels = batch
    # Telling the model not to compute or store gradients, saving memory and speeding up validation
    with torch.no_grad():
      # Forward pass, calculate logit predictions
      logits = model(b_tokenized_texts, token_type_ids=None, attention_mask=b_input_mask)
    
    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    print(logits)
    print(label_ids)
    if batch%10==0:
      break

    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    #tmp_eval_f1 = f1_score(label_ids,logits)
    
    eval_accuracy += tmp_eval_accuracy
    #eval_f1 += tmp_eval_f1
    nb_eval_steps += 1

  print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
  #print("Validation f1: {}".format(eval_f1/nb_eval_steps))
  with open("models/bert.pickle","wb") as f:
    pickle.dump(model,f)

  

with open("models/f_bert.pickle","wb") as f:
  pickle.dump(model,f)





Epoch:   0%|          | 0/4 [00:00<?, ?it/s]




ValueError: ignored

In [0]:
model2=open("models/bert.pickle","rb")

In [0]:
model.eval()


# Tracking variables 
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# Evaluate data for one epoch
for batch in validation_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  # Unpack the inputs from our dataloader
  b_tokenized_texts, b_input_mask, b_labels = batch
  # Telling the model not to compute or store gradients, saving memory and speeding up validation
  with torch.no_grad():
    # Forward pass, calculate logit predictions
    logits = model(b_tokenized_texts, token_type_ids=None, attention_mask=b_input_mask)

  
  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  print(logits)
  print(label_ids)
  break
  tmp_eval_accuracy = flat_accuracy(logits, label_ids)
  #tmp_eval_f1 = f1_score(label_ids,logits)
  
  eval_accuracy += tmp_eval_accuracy
  #eval_f1 += tmp_eval_f1
  nb_eval_steps += 1

  #print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
  #print("Validation f1: {}".format(eval_f1/nb_eval_steps))

[[ 1.488819  -1.7009982]
 [ 1.488819  -1.7009981]
 [ 1.488819  -1.7009983]
 [ 1.4888188 -1.7009981]]
[0 0 0 0]


In [0]:
torch.save(model.state_dict(), "models/bert.pth")

In [0]:
model3 = torch.load("models/bert.pth")


In [0]:
model3.eval()

AttributeError: ignored

In [0]:
model.train()
for _ in trange(int(NUM_TRAIN_EPOCHS), desc="Epoch"):
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(tqdm_notebook(train_dataloader, desc="Iteration")):
        batch = tuple(t.to(device) for t in batch)
        input_ids, input_mask, segment_ids, label_ids = batch

        logits = model(input_ids, segment_ids, input_mask, labels=None)

 
        loss_fct = CrossEntropyLoss()
        loss = loss_fct(logits.view(-1, 2), label_ids.view(-1))
        

        if GRADIENT_ACCUMULATION_STEPS > 1:
            loss = loss / GRADIENT_ACCUMULATION_STEPS

        loss.backward()
        print("\r%f" % loss, end='')
        
        tr_loss += loss.item()
        nb_tr_examples += input_ids.size(0)
        nb_tr_steps += 1
        if (step + 1) % GRADIENT_ACCUMULATION_STEPS == 0:
            optimizer.step()
            optimizer.zero_grad()
            global_step += 1